In [1]:
import pandas as pd
import numpy as np

In [3]:
artists = pd.read_csv('Copy of explicit_data - Artists - All.csv')
artists.head()

,artist_id,artist
0,123,6th Lane
1,63,ajith muthukumarana
2,1,Amarasiri Peris
3,73,Amasha Tissera
4,48,Amila Nidahasa


In [7]:
user_listening = pd.read_csv('copy of explicit_data - DATA preprocessing - artist.csv')
user_listening.head()

,session_id,user_id,age_group,gender,profession,hours_spending,preferred_artists,artist_id,musical_aspect,Unnamed: 9
0,10001,10001,18 - 24,Female,Student,0 - 1,Amarasiri Peris,1,Tempo/speed,NaN
1,10002,10001,18 - 24,Female,Student,0 - 1,BnS,79,Tempo/speed,NaN
2,10003,10001,18 - 24,Female,Student,0 - 1,Umariya Sinhawansha,3,Tempo/speed,NaN
3,10004,10001,18 - 24,Female,Student,0 - 1,Daddy,4,Tempo/speed,NaN
4,10005,10001,18 - 24,Female,Student,0 - 1,Kasun Kalhara,5,Tempo/speed,NaN


In [6]:
songs_with_artist_id = pd.read_csv('Copy of explicit_data - Songs - All-with artist_id.csv')
songs_with_artist_id.head()

,song_id,Title,Artist,Artist_id,Album,Genre
0,2,Aa Ra Sulan,Nirosha Virajini,21,No Album,NaN
1,283,Aale katha,"Kalpana Nayanamadu, Shermaine Willis ft Iraj",11,NaN,NaN
2,3,Ada Nam Ma Hada Iwasum Na,Raveen Kanishka & Kalpana Kavindi,101,NaN,NaN
3,4,Ada Thaniyen Ma Hadanne Na Ma,Shihan Mihiranga,62,NaN,NaN
4,5,Adambarai Baluwama Nam,Surani De Mel,80,NaN,NaN


In [8]:
# merge the two files
user_artist_pref = pd.merge(user_listening, artists.drop_duplicates(['artist_id']), on="artist_id", how="left")
user_artist_pref.head()

,session_id,user_id,age_group,gender,profession,hours_spending,preferred_artists,artist_id,musical_aspect,Unnamed: 9,artist
0,10001,10001,18 - 24,Female,Student,0 - 1,Amarasiri Peris,1,Tempo/speed,NaN,Amarasiri Peris
1,10002,10001,18 - 24,Female,Student,0 - 1,BnS,79,Tempo/speed,NaN,BnS
2,10003,10001,18 - 24,Female,Student,0 - 1,Umariya Sinhawansha,3,Tempo/speed,NaN,Umariya Sinhawansha
3,10004,10001,18 - 24,Female,Student,0 - 1,Daddy,4,Tempo/speed,NaN,Daddy
4,10005,10001,18 - 24,Female,Student,0 - 1,Kasun Kalhara,5,Tempo/speed,NaN,Kasun Kalhara


In [9]:
user_artist_pref = user_artist_pref.drop(['preferred_artists','musical_aspect','Unnamed: 9'], axis=1)
user_artist_pref.head()

,session_id,user_id,age_group,gender,profession,hours_spending,artist_id,artist
0,10001,10001,18 - 24,Female,Student,0 - 1,1,Amarasiri Peris
1,10002,10001,18 - 24,Female,Student,0 - 1,79,BnS
2,10003,10001,18 - 24,Female,Student,0 - 1,3,Umariya Sinhawansha
3,10004,10001,18 - 24,Female,Student,0 - 1,4,Daddy
4,10005,10001,18 - 24,Female,Student,0 - 1,5,Kasun Kalhara


In [147]:
user_songs = pd.read_csv('Copy of explicit_data - Data preprocessing - songs.csv')
songs = pd.read_csv('Copy of explicit_data - Songs - All-with artist_id.csv')

In [148]:
# merge the two files
user_song_pref = pd.merge(user_songs, songs.drop_duplicates(['song_id']), on="song_id", how="left")

In [149]:
user_song_pref = user_song_pref.drop(['Album','Genre'], axis=1)
user_song_pref.head()

,user_id,age_group,gender,profession,hours_spending,musical_aspect,song_id,Title,Artist,Artist_id
0,10001,18 - 24,Female,Student,0 - 1,Tempo/speed,174,Sandaganawa,Dhanith Sri,8
1,10001,18 - 24,Female,Student,0 - 1,Tempo/speed,220,Api hagum walata ida dee mohothak,Victor Rathnayaka,61
2,10002,25 - 34,Male,Working,2 - 3,The singer's voice,221,Mandaram Wahi Watena,6th Lane,123
3,10003,18 - 24,Female,Software Engineer,more than 5,The singer's voice,164,Ru Sara,Bathiya & Santhush,79
4,10004,18 - 24,Male,software engineer,more than 5,The singer's voice,6,Adanne Ay Sudu Manike,H. R. Jothipala,81


# Suggesting songs based on user artist choices

In [153]:
class recommend_songs_artists:
    def __init__(self):
        self.user_songs_data = None
        self.user_artists_data = None
        self.user_id = None
        self.item_id = None
        self.artist_id = None
    
    def create_score(self, user_songs_data):
        train_data_grouped = user_songs_data.groupby(['song_id','Artist_id','Title']).agg({'user_id': 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
        # sort the values to get an overview of the popular songs
        train_data_sort = train_data_grouped.sort_values(['score', 'song_id'], ascending = [0,1])
        train_data_sort = pd.DataFrame(train_data_sort)
        train_data_sort.reset_index(drop=True, inplace = True)
        return train_data_sort
    
    def recommend_songs(self, user_id, user_artists_data, user_songs_data):
        songs = []
        scores = []
        l2 = []
        l3 = []
        user_pref = user_artists_data.loc[user_artists_data['user_id'] == user_id]
        createScore = create_score(user_songs_data)
        for i in user_pref['artist_id']:
            songs.append(createScore['Title'].loc[createScore['Artist_id'] == i])
            scores.append(createScore['score'].loc[createScore['Artist_id'] == i])
        for i in range(len(songs)):
            for j in songs[i]:
                l2.append(j)
        for i in range(len(scores)):
            for j in scores[i]:
                l3.append(j)
        list_of_tuples = list(zip(l2, l3)) 
        df = pd.DataFrame(list_of_tuples, columns = ['Songs', 'Score'])  
        sort_by_life = df.sort_values('Score', ascending=False)
        sort_by_life.reset_index(drop=True, inplace = True)
        return sort_by_life.head(10)

In [155]:
rs = recommend_songs_artists()
rs.recommend_songs(
    user_id = 10002, 
    user_artists_data = user_artist_pref, 
    user_songs_data = user_song_pref) # need history data



,Songs,Score
0,Chandrayan Pidu Kiranak Sagawala Horen,5
1,Landune,4
2,Hitha Hiriwetunado,4
3,Mage punchi rosa male,3
4,Labhadiye,3
5,Malak Une Ai Numba Mata,3
6,Hanthanata Payana Sanda,3
7,Mata Rawana,2
8,Hawasaka Ma,2
9,Sithin Ma Noseli,2


In [156]:
# # get the count of artist_id values
# artists_grouped = user_artist_pref.groupby(['artist']).agg({'artist_id': 'count'}).reset_index()
# # sort the values to get an overview of the popular songs
# artists_grouped.sort_values(['artist_id', 'artist'], ascending = [0,1])

In [157]:
# song_grpd_with_artists = songs_with_artist_id.groupby(['Artist']).agg({'Artist_id': 'count'}).reset_index()
# song_grpd_with_artists.sort_values(['Artist_id', 'Artist'], ascending = [0,1])

In [158]:
# for i in user_artist_pref['user_id']:
#     if i==10001:
#         print(user_artist_pref['artist'])
#         break

In [159]:
# # get the count of song_id values
# train_data_grouped = user_song_pref.groupby(['song_id','Artist_id','Title']).agg({'user_id': 'count'}).reset_index()
# train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
# # sort the values to get an overview of the popular songs
# train_data_sort = train_data_grouped.sort_values(['score', 'song_id'], ascending = [0,1])